# Import Library yang Digunakan

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

# Data Wrangling

## Gathering Data

In [ ]:
raw_data_path = os.path.join(os.getcwd(), '..', '..', 'data', 'raw_data.csv')

In [ ]:
df = pd.read_csv(raw_data_path)
df.head()

In [ ]:
print("Dataset Bukit Vista memiliki {} baris dan {} kolom".format(df.shape[0], df.shape[1]))

## Assesing Data

In [ ]:
df.info()

Disini, kami melihat info terkait nama kolom pada dataset beserta tipe data dari masing-masing kolom. Terlihat bahwa terdapat beberapa kolom yang memiliki tipe data yang salah diantaranya:
1. bathroom
2. beds
3. guests
4. booking_received_timestamp
5. booking_check_in
6. booking_check_out

In [ ]:
print('Jumlah missing value pada tiap kolom:')
df.isnull().sum()

Disini, kami melihat bahwa terdapat beberapa kolom yang memiliki missing value diantaranya:
1. bathroom
2. beds
3. guests
4. lat
5. lng
6. distance_to_coastline
7. review_sentiment_score
8. rating
9. avg_review_sentiment_score

In [ ]:
plt.figure(figsize=(8, 5))
sns.heatmap(df.isnull())
plt.title('Heatmap Missing Value')
plt.show()

Disini, kami melakukan visualisasi terkait missing value untuk mendapatkan gambaran secara visual.

In [ ]:
df.describe()

Disini, kami melihat statistik dari dataframe dengan kolom berisi data numerik.

In [ ]:
for column_name in df.columns:
    # Get distinct values for each column
    distinct_values = df[column_name].unique()
    
    # Print column name and distinct values
    print("Column Name:", column_name)
    print("Distinct Values:", distinct_values, "\n")

Disini, kami melihat distinct value pada setiap kolom.

In [ ]:
print("Jumlah data duplikat pada dataframe adalah {}".format(df.duplicated().sum()))

Pada dataset Bukit Vista, memiliki jumlah data duplikat sebanyak 8.043.

## Cleaning Data

In [ ]:
# List of column names to drop
columns_to_drop = ['unit_id', 
                   'essentials', 
                   'property_bedrooms',
                   'booking_received_timestamp',
                   'booking_check_in',
                   'booking_check_out',
                   'distance_to_coastline',
                   'review_sentiment_score',
                   'booking_earned']

# Drop the specified columns
df.drop(columns=columns_to_drop, inplace=True)

Kami melakukan drop pada beberapa kolom yang ada pada dataframe dengan alasan sebagai berikut:
1. unit_id: melakukan prediksi hanya menggunakan property name dan room_id 
2. essentials: valuenya hanya angka 1
5. property_bedrooms: melakukan prediksi hanya menggunakan property name dan room_id
6. booking_received_timestamp: telah digunakan untuk melakukan feature engineering bernama kolom booking_window
7. booking_check_in: telah digunakan untuk melakukan feature engineering bernama kolom booking_window dan stay_duration_in_days
8. booking_check_out: telah digunakan untuk melakukan feature engineering bernama kolom stay_duration_in_days 
9. booking_earned: telah digunakan untuk melakukan feature engineering bernama kolom average_daily_rate

In [ ]:
# from fancyimpute import IterativeImputer
# mice_imputer = IterativeImputer()

# columns_to_impute = ['bathroom', 
#                      'beds', 
#                      'guests',
#                      'lat',
#                      'lng',
#                      'rating',
#                      'avg_review_sentiment_score']

# # Perform imputation only on selected columns
# imputed_values = mice_imputer.fit_transform(df[columns_to_impute])

# # Create a DataFrame from imputed values
# imputed_df = pd.DataFrame(imputed_values, columns=columns_to_impute)

# # Replace original columns with imputed values
# df[columns_to_impute] = imputed_df

from sklearn.impute import SimpleImputer

# Separate numeric columns
numeric_df = df.select_dtypes(include=['number'])

# Separate non-numeric columns
non_numeric_df = df.select_dtypes(exclude=['number'])

# Initialize the SimpleImputer with mean strategy
imputer = SimpleImputer(strategy='mean')

# Apply the imputer to the numeric dataframe
imputed_numeric_df = pd.DataFrame(imputer.fit_transform(numeric_df), columns=numeric_df.columns, index=numeric_df.index)

# Combine the numeric and non-numeric dataframes
df = pd.concat([non_numeric_df, imputed_numeric_df], axis=1)

df.shape

Kami melakukan imputasi data menggunakan fancy impute atau MICE. MICE adalah Multiple Imputation by Chained Equations. Alasan kami menggunakan MICE dikarenakan missing value pada dataset terlalu banyak.

In [ ]:
# Convert float columns to int
df['bathroom'] = df['bathroom'].astype(int)
df['beds'] = df['beds'].astype(int)
df['guests'] = df['guests'].astype(int)

# Convert object columns to datetime
#df['booking_received_timestamp'] = pd.to_datetime(df['booking_received_timestamp'])
# df['booking_check_in'] = pd.to_datetime(df['booking_check_in'])
# df['booking_check_out'] = pd.to_datetime(df['booking_check_out'])

Berdasarkan info yang telah kami peroleh, maka kami mengubah tipe data pada beberapa kolom. 

In [ ]:
df.drop_duplicates(inplace=True)
print("Jumlah data duplikat pada dataframe adalah {}".format(df.duplicated().sum()))

Kami menghapus data duplikat pada dataframe. Dapat dilihat bahwa jumlah data duplikat saat ini berjumlah 0 baris.

In [ ]:
negative_average_daily_rate = df[df['average_daily_rate'] < 0]

negative_average_daily_rate.head()

Kami menemukan bahwa terdapat beberapa baris yang memiliki nilai average_daily_rate negatif. Average_daily_rate merupakan jumlah uang yang didapatkan dalam penyewaan properti per-hari. Oleh karena itu, tidak memungkinkan bahwa average_daily_rate tersebut bernilai negatif.

In [ ]:
# Directly changing negative values to positive in the original DataFrame
df.loc[df['average_daily_rate'] < 0, 'average_daily_rate'] = df['average_daily_rate'].abs()

print("Jumlah row dengan negative average daily rate saat ini :", df[df['average_daily_rate'] < 0].shape[0])

Sehingga, kami mengubah beberapa baris yang memiliki nilai average_daily_rate yang semula negatif menjadi positif.

In [ ]:
zero_booking_earned = df[df['average_daily_rate'] == 0]

zero_booking_earned.head()

Kami menemukan bahwa terdapat beberapa baris yang memiliki nilai average_daily_rate nol. Average_daily_rate merupakan jumlah uang yang didapatkan dalam penyewaan properti per-hari. Oleh karena itu, tidak memungkinkan bahwa average_daily_rate tersebut bernilai nol.

In [ ]:
df = df[df['average_daily_rate'] != 0]

In [ ]:
print("Jumlah row dengan nol booking earned saat ini :", df[df['average_daily_rate'] == 0].shape[0])

Dapat dilihat bahwa jumlah row dengan nol booking earned saat ini berjumlah nol.

In [ ]:
negative_booking_window = df[df['booking_window'] < 0]

negative_booking_window.shape

Kami melakukan pengecekan baris booking_window yang bernilai negatif. Terlihat bahwa baris booking_window yang bernilai negatif sebanyak 5 baris.

In [ ]:
df = df[df['booking_window'] >= 0]

In [ ]:
print("Jumlah row dengan negatif booking window saat ini :", df[df['booking_window'] < 0].shape[0])

Kami menghapus baris pada dataframe yang memiliki nilai booking_window negatif.

In [ ]:
print("Dataset Bukit Vista setelah preprocessing memiliki {} baris dan {} kolom".format(df.shape[0], df.shape[1]))

Setelah dilakukan preprocessing, dataset Bukit Vista saat ini memiliki jumlah baris sebanyak 58.916 dan jumlah kolom sebanyak 40.

## Exploratory Data Analysis

In [ ]:
# cari stay in day biasanya berapa
stay_duration_count = df['stay_duration_in_days'].value_counts().sort_index()

# buat plotnya
plt.figure(figsize=(10, 6))
stay_duration_count.plot(kind='bar', color='blue', edgecolor='black')
plt.xlabel('Stay Duration in Days')
plt.ylabel('Frequency')
plt.title('Frequency of Stay Duration in Days')
plt.xticks(rotation=45)
plt.show()

Disini kami mencari tahu pada umumnya, seberapa lama tamu menginap pada properti-properti yang dikelola oleh Bukit Vista. Dapat dilihat bahwa kebanyakan tamu menginap selama 2 hari.

In [ ]:
 # buat plotnya
plt.figure(figsize=(15, 6))
plt.hist(df['booking_window'], bins=20, color='blue', edgecolor='black')
plt.xlabel('Days')
plt.ylabel('Frequency')
plt.title('Booking Window in Days')
plt.xticks(rotation=45)
plt.show()

Disini, kami mencari tahu berapa frekuensi booking window dengan histogram yang memiliki 20 bins. Dapat dilihat bahwa data condong ke kanan.

In [ ]:
# Create a new DataFrame with the rounded rating column
df_rounded = df.copy()
df_rounded['rating_rounded'] = df['avg_rating'].round(1)

# Count the occurrences of each rounded rating
rating_counts = df_rounded['rating_rounded'].value_counts().sort_index()

# Plot the bar chart
plt.figure(figsize=(10, 6))
rating_counts.plot(kind='bar', color='blue', edgecolor='black')

# Set labels and title
plt.xlabel('Rating')
plt.ylabel('Count')
plt.title('Distribution of Ratings')
plt.xticks(rotation=0)  # Ensure x-axis labels are horizontal

# Show the plot
plt.show()

Disini, kami mencari tahu sebaran dari kolom rating. Dapat dilihat bahwa kebanyakan tamu menilai properti-properti yang dikelola oleh Bukit Vista dengan nilai 5.

In [ ]:
# Menggambarkan korelasi antara penilaian properti dengan pemasukkan properti
rating_booking = df.groupby('avg_rating')['average_daily_rate'].sum().reset_index()

rating_booking = rating_booking.sort_values(by='avg_rating')

plt.figure(figsize=(10, 6))
plt.bar(rating_booking['avg_rating'], rating_booking['average_daily_rate'], color='blue', edgecolor='black')
plt.xlabel('Rating')
plt.ylabel('Average Daily Rate')
plt.title('Average Daily Rate by Rating')
plt.show()

Disini, kami membuat diagram batang untuk mencari tahu apakah ada relasi antara penilaian properti dengan rata-rata uang yang didapat oleh properti tersebut per-hari. Dapat dilihat bahwa terdapat korelasi antara penilaian properti dengan rata-rata uang yang didapat oleh properti tersebut per-hari.

In [ ]:
# visualisasi lat dan lng
plt.figure(figsize=(10, 6))
plt.scatter(df['lng'], df['lat'], c='blue', alpha=0.5, edgecolors='w', s=50)

# Set labels and title
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Geographical Scatter Plot of Coordinates')

# Show the plot
plt.show()

Disini kami mencoba untuk memvisualisasikan koordinat yang ada pada kolom "lat" dan "lng".

In [ ]:
# Count the occurrences of each property name and sort them
type_counts = df['property_name'].value_counts().sort_values()

# Get the top 3 and bottom 3 room types
top_3 = type_counts[-3:]
bottom_3 = type_counts[:3]

# Concatenate top 3 and bottom 3
top_bottom_3 = pd.concat([bottom_3, top_3])

# Plot the bar chart
plt.figure(figsize=(10, 6))
top_bottom_3.plot(kind='barh', color='blue', edgecolor='black')  # Use horizontal bar chart

# Set labels and title
plt.xlabel('Count')
plt.ylabel('Property Name')
plt.title('Top 3 and Bottom 3 Property Name by Visit Count')

# Show the plot
plt.show()

Disini kami memvisualisasikan perbedaan jumlah pengunjung berdasarkan nama ruangan, dan menampilkan hanya berdasarkan 3 terbanyak dan 3 terendah.

In [ ]:
num_columns = len(df.columns)
num_rows = (num_columns + 2) // 3

plt.figure(figsize=(15, 5 * num_rows))

for i, column in enumerate(df.select_dtypes(include=['number'])):
    plt.subplot(num_rows, 3, i + 1)
    sns.boxplot(x=df[column], color="blue")
    plt.title(f'Box Plot for {column}')

plt.tight_layout()
plt.show()

Disini, kami melakukan visualisasi menggunakan boxplot untuk mencari outlier.

In [ ]:
numeric_columns = df.select_dtypes(include=['number'])

plt.figure(figsize=(25,20))
sns.heatmap(numeric_columns.corr(), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Korelasi Heatmap dari kolom-kolom numerik')
plt.show()

Disini, kami memvisualisasikan korelasi antara semua kolom numerik, dan diberi warna berdasarkan tingkat korelasinya menggunakan heatmap.

## Save Data into CSV

In [ ]:
# cara mendownload data hasil preprocessing :
path = os.path.join(os.path.dirname(raw_data_path), 'preprocessed_data.csv')
df.to_csv('preprocessed_data.csv', index=False)

Kami menyimpan data hasil preprocessing dengan nama preprocessed_data.csv